# A note about opening notebooks in shared workspaces <a class="tocSkip">

Please do not run or edit master copies of notebooks unless you intend to improve the code. As a general rule, it is good to be cautious when editing a notebook in a shared workspace, because you don't want to overwrite the work of your collaborators. Best practices is to test in a cloned workspace or duplicate notebook with an easily identifiable name.  

# What is BigQuery? <a class="tocSkip">
BigQuery is the Google Cloud Storage solution for structured data (like a spreadsheet optimized for quick retieval of particular sections that you access with a "query". To learn more, see this five-minute video from Google here. Many datasets, including the public-access 1,000 Genomes Project, are stored in BigQuery, for anyone to access.

# Notebook overview <a class="tocSkip">

## 1,000 Genomes via bigrquery and dplyr <a class="tocSkip">

This notebook demonstrates two ways to access BigQuery in an R-based notebook:
1. Using SQL syntax
2. Using only R code to extract the data of interest from BigQuery

# R-environment setup

First, be sure to run the general notebook **`R environment setup`** in this workspace. 

Then run the cells below to add additional needed libraries, set the project id, and authorize the BigQuery client. Details about what each individual code cell does are included in green comments for reference. 

**Note** that when you run these cells, output in a red box does not signify that the code is broken, and will not affect how the notebook runs.

In [ ]:
# Load additional R libraries needed for this notebook into memory
# There may be warnings in pink indicating that objects are 'masked'.  These warnings can be ignored. 

# The bigrquery package makes it easy to work with data stored in Google BigQuery by allowing you to query 
# BigQuery tables and retrieve metadata about your projects, datasets, tables, and jobs
library(bigrquery)

# ggplot2 is a library of integrated plotting functions
library(ggplot2)

# dplyr is an R package for working with structured data (like BigQuery) both in and outside of R
library(dplyr)

# skimr handles different data types and returns a skim_df object which can be included in a 
# tidyverse pipeline or displayed nicely for the human reader
library(skimr)

In [ ]:
# Set the project id of the clould project to bill for queries to BigQuery
# Note that although you do pay a fee for the query in this notebook, it is very small (cents)
BILLING_PROJECT_ID <- Sys.getenv('GOOGLE_PROJECT')

In [ ]:
# To access BigQuery, you must first autheniticate, or verify, your identity.
# The following command does that
bigrquery::set_service_token(Ronaldo::getServiceAccountKey())

# Retrieve filtered data
Now that we've set up the virtual compute environment, BigQuery authentication and billing, let’s retrieve a subset of fields and samples metadata in the [1000 Genomes](http://www.internationalgenome.org/data "1000 Genomes") dataset.

We will do this in two different ways. Both return the same results.

1. Using bigrquery (standard SQL)
2. Using dplyr and dbplyr

## Option 1: Using bigrquery (SQL)

To learn more about SQL syntax see the [BigQuery standard SQL reference](https://cloud.google.com/bigquery/docs/reference/standard-sql/).

In [ ]:
# Run a query from the public-access 1,000 genomes data in BigQuery with standard SQL 
phase1_samples_tbl <- bigrquery::bq_project_query(
    BILLING_PROJECT_ID,
    query = '
SELECT
  Sample,
  Gender,
  Relationship,
  Population,
  Population_Description,
  Super_Population,
  Super_Population_Description,
  Total_Exome_Sequence,
  Main_Project_E_Platform,
  Main_Project_E_Centers
FROM
  `bigquery-public-data.human_genome_variants.1000_genomes_sample_info`
WHERE
  -- Only include information for samples in phase 1.
  In_Phase1_Integrated_Variant_Set = TRUE
  -- Only include information for samples with these relationships.
  AND Relationship IN ("", "unrel", "child", "father", "mother",
  "mat grandmother", "pat grandmother",
  "mat grandfather", "pat grandfather")
')

phase1_samples <- bigrquery::bq_table_download(phase1_samples_tbl)

The next cell serves as a sanity check. It should show that 1086 rows and 10 columns were retrieved into the "phase1_samples" table.

In [ ]:
# Print out the dimensions of the table
dim(phase1_samples)

The next cell provides summary statistics of the samples.

In [ ]:
print(skim(phase1_samples))

In [ ]:
# Plot the relationship of individuals contained in the data set
# The null condition (first column) indicates that no individuals were related to this person
ggplot(data = phase1_samples,
       aes(x=Relationship, fill=Relationship)) +
  geom_bar() +
  theme(axis.text.x = element_text(angle = 50, hjust = 1, vjust = 1))

## Option 2: Using dplyr

**dplyr** is an R package that provides a common set of "verbs" to manipulate tabular data.

To learn more about dplyr see [R for Data Science Chapter 5 Data transformation](http://r4ds.had.co.nz/transform.html "Chapter 5").

In [ ]:
# Create a "connection" to a public BigQuery dataset
dbcon <- bigrquery::src_bigquery(project = 'bigquery-public-data',
                                 dataset = 'human_genome_variants',
                                 billing = BILLING_PROJECT_ID)

# Create a 'virtual dataframe' backed by a BigQuery table
sample_info <- dplyr::tbl(dbcon, '1000_genomes_sample_info')

In [ ]:
# Filter to only include information for samples in phase 1 with the relationshionships below
phase1_only <- sample_info %>% filter(
    In_Phase1_Integrated_Variant_Set == TRUE,
    Relationship %in% c(
      '', 'unrel', 'child', 'father', 'mother',
      'mat grandmother', 'pat grandmother',
      'mat grandfather', 'pat grandfather')
  )
# Nothing is computed on BigQuery yet.

In [ ]:
# Define a variable, "sample_fields", with the included parameters
sample_fields <- phase1_only %>% select(
  Sample,
  Gender,
  Relationship,
  Population,
  Population_Description,
  Super_Population,
  Super_Population_Description,
  Total_Exome_Sequence,
  Main_Project_E_Platform,
  Main_Project_E_Centers
  )
# Still nothing computed on BigQuery.

In [ ]:
# Optional sanity check: Take a look at the SQL
dplyr::show_query(sample_fields)

In [ ]:
# Optional: See how much data this will return
sample_fields %>% summarize(cnt = n()) %>% collect()

In [ ]:
# Execute the query and return all results into an in-memory table in R
phase1_samples <- sample_fields %>% collect()

In [ ]:
# Sanity check: What are the dimensions of the phase1_sample table?
dim(phase1_samples)

In [ ]:
# Plot results
ggplot(data = phase1_samples,
       aes(x=Relationship, fill=Relationship)) +
  geom_bar() +
  theme(axis.text.x = element_text(angle = 50, hjust = 1, vjust = 1))

# Provenance

In [ ]:
# Including this information allows you to easly go back and see the details of your notebook environment
# Provenance is also recommended as best practices for reproducible research
devtools::session_info()

Copyright 2019 The Broad Institute, Inc., Verily Life Sciences, LLC All rights reserved.

This software may be modified and distributed under the terms of the BSD license. See the LICENSE file for details.